In [1]:
# import all dependencies
import datetime as dt
import yfinance as yf
from plotly.offline import plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# time horizon set-up
start = dt.datetime.today() - dt.timedelta(365) # 1 year time-horizon
end = dt.datetime.today()

In [4]:
# getting data
stock = 'AAPL'
ohlcv = yf.download(stock, start, end)

[*********************100%***********************]  1 of 1 completed


In [5]:
# macd function
def macd(df, fast_periods, slow_periods, signal_periods):
    df = df.copy()
    df['ma_fast'] = df['Adj Close'].ewm(span=fast_periods, min_periods=fast_periods).mean()
    df['ma_slow'] = df['Adj Close'].ewm(span=slow_periods, min_periods=slow_periods).mean()
    df['macd'] = df['ma_fast'] - df['ma_slow']
    df['signal'] = df['macd'].ewm(span=signal_periods, min_periods=signal_periods).mean()
    df['signal'].dropna(inplace=True)
    return df

In [6]:
# macd
df_macd = macd(ohlcv, 12, 26, 9)

In [7]:
df_macd

,Open,High,Low,Close,Adj Close,Volume,ma_fast,ma_slow,macd,signal
Date,,,,,,,,,,
2019-12-13,67.864998,68.824997,67.732498,68.787498,67.644707,133587600,NaN,NaN,NaN,NaN
2019-12-16,69.250000,70.197502,69.245003,69.964996,68.802635,128186000,NaN,NaN,NaN,NaN
2019-12-17,69.892502,70.442497,69.699997,70.102501,68.937851,114158400,NaN,NaN,NaN,NaN
2019-12-18,69.949997,70.474998,69.779999,69.934998,68.773132,116028400,NaN,NaN,NaN,NaN
2019-12-19,69.875000,70.294998,69.737503,70.004997,68.841965,98369200,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-12-07,122.309998,124.570000,122.250000,123.750000,123.750000,86712000,120.428583,118.697414,1.731169,1.108582
2020-12-08,124.370003,124.980003,123.089996,124.379997,124.379997,82225500,121.036493,119.118346,1.918146,1.270495
2020-12-09,124.529999,125.949997,121.000000,121.779999,121.779999,115089200,121.150878,119.315506,1.835373,1.383470


In [11]:
# create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_macd.index, y=df_macd['Adj Close'], name='price'))
fig.add_trace(go.Scatter(x=df_macd.index, y=df_macd['macd'], name='macd'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_macd.index, y=df_macd['signal'], name='signal'), secondary_y=True)
fig.add_trace(go.Scatter(x=[df_macd.index[0], df_macd.index[-1]], y=[0, 0], name='reference'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'MACD of {stock}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')
fig.update_yaxes(title_text='MACD & Signal', secondary_y=True)